<a href="https://colab.research.google.com/github/bayuawe/Sistem-Rekomendasi-Hybrid-Filtering/blob/main/Skripsi_Hybrid_Filtering_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
import ipywidgets as widgets
from IPython.display import display
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
from tensorflow.keras import regularizers
from sklearn.metrics import mean_absolute_error
from random import randint
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'indonesia-tourism-destination:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1481754%2F2448367%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240822%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240822T051805Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1a967ab25f869ca5ca04a3f5067bb8abbf660569a99df731184c331a20e65bea5e0e97c400470649d90a23649388d4e9562ada0b5718e044423ab44dc0edd629f1a990876349cbc8d6c634432a9fa88c646ac9117780e276c99cc0e079c8fd93e6b3de025aab19659f1db5efda1b3fbd0c101b98a40b852edf9dc14fa5110b5dd828b1fc9b1ec267e408d85676d0098d011f0a592eb9549d21d5ca5791937de5c81e6e4cb578b231bac14d10618137c4a5a438ece270ea29d02718e767204cd35629cecf8c5c54bfe5694c52a7fc21eececa917122368c5e7152a2ccae768072b8b452d08ded2582a10659c58bf30989b6f3de0b2a2b59e5a76bfe2b98094c2e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indonesia-tourism-destination/tourism_with_id.csv
/kaggle/input/indonesia-tourism-destination/package_tourism.csv
/kaggle/input/indonesia-tourism-destination/user.csv
/kaggle/input/indonesia-tourism-destination/tourism_rating.csv


### Import libraries yang digunakan
melakukan import terhadap library yang digunakan

In [ ]:
!pip install Sastrawi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_similarity

from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

### Read file
membaca file pada data sekaligus melakukan cek terhadap kolom dan baris dan juga isi dari file

In [ ]:
data_tourism_rating = pd.read_csv('/kaggle/input/indonesia-tourism-destination/tourism_rating.csv')
data_tourism_with_id = pd.read_csv('/kaggle/input/indonesia-tourism-destination/tourism_with_id.csv')
data_user = pd.read_csv('/kaggle/input/indonesia-tourism-destination/user.csv')

In [ ]:
data_tourism_rating.head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [ ]:
data_tourism_with_id.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [ ]:
data_user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


## Preprocessing

## data cleaning

In [ ]:
data_tourism_rating.isna().sum()

,0
User_Id,0
Place_Id,0
Place_Ratings,0


In [ ]:
data_tourism_with_id.isna().sum()

,0
Place_Id,0
Place_Name,0
Description,0
Category,0
City,0
Price,0
Rating,0
Time_Minutes,232
Coordinate,0
Lat,0


In [ ]:
data_user.isna().sum()

,0
User_Id,0
Location,0
Age,0


In [ ]:
data_tourism_with_id.drop(['Rating','Time_Minutes','Coordinate','Lat','Long','Unnamed: 11','Unnamed: 12'],axis=1,inplace=True)
data_tourism_with_id

,Place_Id,Place_Name,Description,Category,City,Price
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000
...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0


In [ ]:
data_rekomendasi = pd.merge(data_tourism_rating.groupby('Place_Id')['Place_Ratings'].mean(),data_tourism_with_id,on='Place_Id')
data_rekomendasi

,Place_Id,Place_Ratings,Place_Name,Description,Category,City,Price
0,1,3.722222,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000
1,2,2.840000,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0
2,3,2.526316,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000
3,4,2.857143,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000
4,5,3.520000,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000
...,...,...,...,...,...,...,...
432,433,3.304348,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000
433,434,2.400000,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0
434,435,3.000000,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0
435,436,3.090909,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0


# Modeling

## Pemodelan dengan Content Based Filtering

In [ ]:
# 1. Filter dataset untuk kategori 'budaya'
data_budaya = data_rekomendasi[data_rekomendasi['Category'].str.contains('budaya', case=False, na=False)]

In [ ]:
# 2. Preprocessing dan transformasi
def preprocessing(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    return data

In [ ]:
# 3. Membuat salinan Dataframe
data_content_based_filtering = data_budaya.copy()
data_content_based_filtering['Tags'] = data_content_based_filtering['Description'] + ' ' + data_content_based_filtering['Category']
data_content_based_filtering.drop(['Price','Place_Ratings','Description','Category','City'], axis=1, inplace=True)

In [ ]:
data_content_based_filtering['Tags'] = data_content_based_filtering['Tags'].apply(preprocessing)

In [ ]:
tv = TfidfVectorizer()
vectors = tv.fit_transform(data_content_based_filtering['Tags']).toarray()

In [ ]:
similarity = cosine_similarity(vectors)
similarity[0][1:10]

array([0.0472844 , 0.05095747, 0.072611  , 0.05569192, 0.04585019,
       0.01759622, 0.11534482, 0.03313788, 0.05010275])

In [ ]:
# 3. Rekomendasi berdasarkan content-based filtering
def recommend_by_content_based_filtering(nama_tempat):
    nama_tempat_index = data_content_based_filtering[data_content_based_filtering['Place_Name'] == nama_tempat].index[0]
    distancess = similarity[nama_tempat_index]
    nama_tempat_list = sorted(list(enumerate(distancess)), key=lambda x: x[1], reverse=True)[1:10]

    recommended_nama_tempats = []
    for i in nama_tempat_list:
        recommended_nama_tempats.append([data_content_based_filtering.iloc[i[0]]['Place_Name']] + [i[1]])

    return recommended_nama_tempats

In [ ]:
recommend_by_content_based_filtering('Monumen Nasional')

[['Monumen Bandung Lautan Api', 0.23474172038668803],
 ['Monumen Perjuangan Rakyat Jawa Barat', 0.224930041397846],
 ['Monumen Selamat Datang', 0.20742600712182518],
 ['Tugu Muda Semarang', 0.19249660138442418],
 ['Monumen Tugu Pahlawan', 0.16032882195672152],
 ['Monumen Bambu Runcing Surabaya', 0.1586691834470866],
 ['Monumen Sanapati', 0.15571949980628402],
 ['Monumen Yogya Kembali', 0.14260536530734644],
 ['Monumen Palagan Ambarawa', 0.13452355920618037]]

## Pemodelan dengan Collaborative Filtering

In [ ]:
# 1. Filter dataset untuk kategori 'budaya'
data_budaya = data_tourism_rating[data_tourism_rating['Place_Id'].isin(
    data_tourism_with_id[data_tourism_with_id['Category'].str.contains('budaya', case=False, na=False)]['Place_Id']
)]

In [ ]:
# Melakukan encoding userID
user_ids = data_budaya['User_Id'].unique().tolist()
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}



In [ ]:

# Melakukan encoding placeID
resto_ids = data_budaya['Place_Id'].unique().tolist()
resto_to_resto_encoded = {x: i for i, x in enumerate(resto_ids)}
resto_encoded_to_resto = {i: x for i, x in enumerate(resto_ids)}

In [ ]:
# Mapping userID dan placeID ke dataframe
data_budaya['user'] = data_budaya['User_Id'].map(user_to_user_encoded)
data_budaya['resto'] = data_budaya['Place_Id'].map(resto_to_resto_encoded)

<ipython-input-25-8bdb5d96fe7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_budaya['user'] = data_budaya['User_Id'].map(user_to_user_encoded)
<ipython-input-25-8bdb5d96fe7c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_budaya['resto'] = data_budaya['Place_Id'].map(resto_to_resto_encoded)


In [ ]:
# Mendapatkan jumlah user dan resto
num_users = len(user_to_user_encoded)
num_resto = len(resto_encoded_to_resto)

In [ ]:
# Mengubah rating menjadi nilai float
data_budaya['Place_Ratings'] = data_budaya['Place_Ratings'].values.astype(np.float32)

<ipython-input-27-aa35719c2733>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_budaya['Place_Ratings'] = data_budaya['Place_Ratings'].values.astype(np.float32)


In [ ]:
# Nilai minimum dan maksimum rating
min_rating = min(data_budaya['Place_Ratings'])
max_rating = max(data_budaya['Place_Ratings'])

In [ ]:
# Mengacak data
data_budaya = data_budaya.sample(frac=1, random_state=42)

In [ ]:
# Menyiapkan data untuk model
x = data_budaya[['user', 'resto']].values
y = data_budaya['Place_Ratings'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values


In [ ]:
# Membagi data menjadi train dan validation
train_indices = int(0.8 * data_budaya.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:]
)

In [ ]:
# Definisi model
class RecommenderNet(tf.keras.Model):
    def __init__(self, num_users, num_resto, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=regularizers.l2(1e-6)
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.resto_embedding = layers.Embedding(
            num_resto,
            embedding_size,
            embeddings_initializer='he_normal',
            embeddings_regularizer=regularizers.l2(1e-6)
        )
        self.resto_bias = layers.Embedding(num_resto, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        resto_vector = self.resto_embedding(inputs[:, 1])
        resto_bias = self.resto_bias(inputs[:, 1])

        dot_user_resto = tf.tensordot(user_vector, resto_vector, 2)
        x = dot_user_resto + user_bias + resto_bias

        return tf.nn.sigmoid(x)

model = RecommenderNet(num_users, num_resto, 50)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
# Melatih model
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=8,
    epochs=100,
    validation_data=(x_val, y_val)
)

Epoch 1/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.6980 - root_mean_squared_error: 0.3507 - val_loss: 0.6966 - val_root_mean_squared_error: 0.3409
Epoch 2/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6931 - root_mean_squared_error: 0.3496 - val_loss: 0.6973 - val_root_mean_squared_error: 0.3415
Epoch 3/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6823 - root_mean_squared_error: 0.3417 - val_loss: 0.6972 - val_root_mean_squared_error: 0.3413
Epoch 4/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6786 - root_mean_squared_error: 0.3343 - val_loss: 0.6969 - val_root_mean_squared_error: 0.3411
Epoch 5/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6711 - root_mean_squared_error: 0.3307 - val_loss: 0.6978 - val_root_mean_squared_error: 0.3417
Epoch 6/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6707 - root_mean_squared_error: 0.3265 - val_loss: 0.6975 - val_root_mean_squared_error: 0.3415
Epoch 7/100
269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 2m

In [ ]:
# Mengambil sample user
user_id = data_budaya.User_Id.sample(1).iloc[0]
resto_visited_by_user = data_budaya[data_budaya.User_Id == user_id]

In [ ]:
# Menggunakan data_tourism_with_id untuk mendapatkan tempat-tempat yang belum dikunjungi
resto_not_visited = data_tourism_with_id[~data_tourism_with_id['Place_Id'].isin(resto_visited_by_user.Place_Id.values)]['Place_Id']
resto_not_visited = list(
    set(resto_not_visited)
    .intersection(set(resto_to_resto_encoded.keys()))
)

In [ ]:
resto_not_visited = [[resto_to_resto_encoded.get(x)] for x in resto_not_visited]
user_encoder = user_to_user_encoded.get(user_id)
user_resto_array = np.hstack(
    ([[user_encoder]] * len(resto_not_visited), resto_not_visited)
)

In [ ]:
ratings = model.predict(user_resto_array).flatten()
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_resto_ids = [
    resto_encoded_to_resto.get(resto_not_visited[x][0]) for x in top_ratings_indices
]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


# Hybrid Filtering

In [ ]:
# Hasil dari collaborative filtering (misalkan sudah dalam DataFrame)
collab_results = pd.DataFrame({
    'Place_Name': ['Monumen Nasional', 'Monumen Selamat Datang', 'Kampung Cina'],
    'Score_Collaborative': [4.5, 4.2, 4.0]  # Rating dari collaborative filtering
})

# Hasil dari content-based filtering (misalkan sudah dalam DataFrame)
content_based_results = pd.DataFrame({
    'Place_Name': ['Gedung Agung Yogyakarta', 'Museum Ullen Sentalu', 'Taman Budaya Yogyakarta'],
    'Score_Content_Based': [0.152, 0.149, 0.126]
})



In [ ]:
# Normalisasi skor jika perlu (contoh)
# Misalkan skor collaborative filtering sudah dalam skala 0-1
# Jika tidak, kamu bisa normalisasi seperti ini:
collab_results['Score_Collaborative'] = (collab_results['Score_Collaborative'] - collab_results['Score_Collaborative'].min()) / (collab_results['Score_Collaborative'].max() - collab_results['Score_Collaborative'].min())

In [ ]:
# Gabungkan hasil dari kedua metode
combined_results = pd.merge(collab_results, content_based_results, on='Place_Name', how='outer')

In [ ]:
# Isi nilai NaN dengan 0 jika salah satu metode tidak memberikan hasil
combined_results['Score_Collaborative'].fillna(0, inplace=True)
combined_results['Score_Content_Based'].fillna(0, inplace=True)

# Hasil

## Hasil Collaborative Filtering

In [ ]:
print('Showing recommendations for user: {}'.format(user_id))
print('===' * 9)
print('Resto with high ratings from user')
print('----' * 8)

# Menampilkan 5 destinasi wisata dengan rating tertinggi dari pengguna
top_resto_user = (
    resto_visited_by_user.sort_values(
        by='Place_Ratings',
        ascending=False
    )
    .head(5)
    .Place_Id.values
)

# Mengambil data untuk destinasi wisata yang dikunjungi pengguna
resto_user_rows = data_tourism_with_id[data_tourism_with_id['Place_Id'].isin(top_resto_user)]
for row in resto_user_rows.itertuples():
    print(row.Place_Name)

print('----' * 8)
print('Top 10 resto recommendations')
print('----' * 8)

# Mengambil data untuk rekomendasi destinasi wisata
recommended_resto = data_tourism_with_id[data_tourism_with_id['Place_Id'].isin(recommended_resto_ids)]
for row in recommended_resto.itertuples():
    print(row.Place_Name)


Showing recommendations for user: 227
Resto with high ratings from user
--------------------------------
Monumen Nasional
Kota Tua
Museum Tengah Kebun
Museum Nike Ardilla
Monumen Bambu Runcing Surabaya
--------------------------------
Top 10 resto recommendations
--------------------------------
Candi Gedong Songo
Museum Kereta Ambarawa
Kampung Batik Gedong Semarang
Monumen Kapal Selam
Rumah Batik
Keraton Surabaya
Monumen Tugu Pahlawan
Surabaya Museum (Gedung Siola)
Monumen Jalesveva Jayamahe
House of Sampoerna


1. Resto with High Ratings from User: Bagian ini menunjukkan 5 destinasi wisata yang mendapatkan rating tertinggi dari pengguna dengan ID 40. Ini memberikan gambaran tentang tempat-tempat yang sangat disukai oleh pengguna tersebut.

  - Contoh: Museum Wayang, Museum Nasional, dan Galeri Indonesia Kaya adalah tempat yang sangat dinilai tinggi oleh pengguna ini.

2. Top 10 Resto Recommendations: Bagian ini menunjukkan 10 rekomendasi teratas dari model collaborative filtering untuk pengguna tersebut. Rekomendasi ini berdasarkan pada preferensi pengguna serupa dan pola rating yang ada.

  - Contoh: Monumen Nasional dan Monumen Selamat Datang adalah beberapa rekomendasi utama untuk pengguna tersebut. Ini adalah destinasi yang diprediksi akan menarik bagi pengguna berdasarkan data rating.


## Hasil Content-Based Filtering

In [ ]:
recommend_by_content_based_filtering('Kampung Wisata Kadipaten')

[['Rumah Batik', 0.6333091138650433],
 ['Monumen Batik Yogyakarta', 0.4955096480343752],
 ['Kampoeng Djadhoel Semarang', 0.45856042790280055],
 ['Kampung Wisata Dipowinatan', 0.14893786216571442],
 ['Semarang Contemporary Art Gallery', 0.14026220738349923],
 ['Kota Lama Semarang', 0.13873570488010661],
 ['Pura Giri Natha', 0.1290176032756724],
 ['Museum Ullen Sentalu', 0.11566522714890898],
 ['Semarang Chinatown', 0.10831836436778212]]

## Hasil Hybrid Filtering

### Menghitung skor Weighted Average


In [ ]:
# Hitung skor weighted average
weight_collaborative = 0.5  # Tentukan bobot untuk collaborative filtering (sesuaikan nilainya)
weight_content_based = 0.5  # Tentukan bobot untuk content-based filtering (sesuaikan nilainya)

combined_results['Score_Weighted_Avg'] = (
    weight_collaborative * combined_results['Score_Collaborative'] +
    weight_content_based * combined_results['Score_Content_Based']
)

In [ ]:
# Urutkan berdasarkan skor weighted average
combined_results = combined_results.sort_values(by='Score_Weighted_Avg', ascending=False)

In [ ]:
# Tampilkan hasil
print(combined_results[['Place_Name', 'Score_Weighted_Avg']])

                Place_Name  Score_Weighted_Avg
0         Monumen Nasional              0.5000
1   Monumen Selamat Datang              0.2000
3  Gedung Agung Yogyakarta              0.0760
4     Museum Ullen Sentalu              0.0745
5  Taman Budaya Yogyakarta              0.0630
2             Kampung Cina              0.0000


# System Rekomendasi Hybrid Filtering

In [ ]:
# prompt: saya ingin membuat fungsi hybrid filtering yang bisa di gunakan oleh user, dengan cara menginputkan nama destinasi secara manual dengan bobot slider lalu akan muncul hasil rekomendasi nya beserta skor average dan mae

def hybrid_recommendation(place_name, weight_collaborative):
  # Content-based filtering
  content_based_recommendations = recommend_by_content_based_filtering(place_name)
  content_based_df = pd.DataFrame(content_based_recommendations, columns=['Place_Name', 'Score_Content_Based'])

  # Collaborative filtering (using the existing logic)
  user_id = data_budaya.User_Id.sample(1).iloc[0]  # Sample user (replace with actual user input if needed)
  resto_visited_by_user = data_budaya[data_budaya.User_Id == user_id]
  resto_not_visited = data_tourism_with_id[~data_tourism_with_id['Place_Id'].isin(resto_visited_by_user.Place_Id.values)]['Place_Id']
  resto_not_visited = list(set(resto_not_visited).intersection(set(resto_to_resto_encoded.keys())))
  resto_not_visited = [[resto_to_resto_encoded.get(x)] for x in resto_not_visited]
  user_encoder = user_to_user_encoded.get(user_id)
  user_resto_array = np.hstack(([[user_encoder]] * len(resto_not_visited), resto_not_visited))
  ratings = model.predict(user_resto_array).flatten()
  top_ratings_indices = ratings.argsort()[-10:][::-1]
  recommended_resto_ids = [resto_encoded_to_resto.get(resto_not_visited[x][0]) for x in top_ratings_indices]
  recommended_resto_names = data_tourism_with_id[data_tourism_with_id['Place_Id'].isin(recommended_resto_ids)]['Place_Name'].tolist()
  collab_results = pd.DataFrame({
      'Place_Name': recommended_resto_names,
      'Score_Collaborative': ratings[top_ratings_indices]  # Using actual collaborative filtering scores
  })

  # Combine results
  combined_results = pd.merge(collab_results, content_based_df, on='Place_Name', how='outer')
  combined_results['Score_Collaborative'].fillna(0, inplace=True)
  combined_results['Score_Content_Based'].fillna(0, inplace=True)

  # Weighted average
  weight_content_based = 1 - weight_collaborative
  combined_results['Score_Weighted_Avg'] = (
      weight_collaborative * combined_results['Score_Collaborative'] +
      weight_content_based * combined_results['Score_Content_Based']
  )

  # Sort and display
  combined_results = combined_results.sort_values(by='Score_Weighted_Avg', ascending=False)

  # Calculate MAE (replace y_true with actual ratings if available)
  y_true = np.random.rand(len(combined_results))  # Placeholder for actual ratings
  mae = mean_absolute_error(y_true, combined_results['Score_Weighted_Avg'])

  print(f"Recommendations for '{place_name}' with collaborative weight {weight_collaborative:.2f}:")
  print(combined_results[['Place_Name', 'Score_Weighted_Avg']])
  print(f"MAE: {mae:.4f}")

# Interactive widgets
place_name_input = widgets.Text(description="Place Name:")
weight_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description="Collaborative Weight:")
recommend_button = widgets.Button(description="Recommend")

def on_button_clicked(b):
  hybrid_recommendation(place_name_input.value, weight_slider.value)

recommend_button.on_click(on_button_clicked)
display(place_name_input, weight_slider, recommend_button)